# Fazendinha Contente
## Trabalho II - Computação Gráfica
### Membros do grupo:
- Beatriz Aparecida Diniz, 11925430
- João Lucas Rodrigues Constantino, 11795763
- Leandro Sena, 9293060
- Tulio Santana, 11795526

# Inicialização da Janela

In [1]:
# Para instalar a API PyWaveFront
!pip install pywavefront

In [2]:
# Dependências gerais
import math

# Dependências locais
from utils.glfw_window import GLFWWindow
from utils.opengl_session import OpenGLSession
from utils.camera import Camera
from utils.components import *
from utils.transformations import *

# Propriedades da janela
WINDOW_WIDTH = 1280
WINDOW_HEIGHT = 720
WINDOW_TITLE = "Fazendinha Contente"

# Inicialização da janela, da sessão e da câmera
window = GLFWWindow(WINDOW_WIDTH, WINDOW_HEIGHT, WINDOW_TITLE)
session = OpenGLSession(window)
camera = Camera(speed = 0.2, last_xpos=WINDOW_WIDTH/2, last_ypos=WINDOW_HEIGHT/2)

# Construção dos Componentes

In [3]:
# Garota
girl = WaveFrontObject("obj/girl/girl OBJ.obj")

# Registro das componentes
session.add_component(girl)

Unimplemented OBJ format statement 's' on line 's 1'


Unimplemented OBJ format statement 's' on line 's 1'
Unimplemented OBJ format statement 's' on line 's 1'
Unimplemented OBJ format statement 's' on line 's 1'
Unimplemented OBJ format statement 's' on line 's off'
Unimplemented OBJ format statement 's' on line 's 1'
Unimplemented OBJ format statement 's' on line 's 1'
Unimplemented OBJ format statement 's' on line 's 1'
Unimplemented OBJ format statement 's' on line 's 1'


# Eventos de teclado e de mouse

In [4]:
# Para aumentar ou diminuir a velocidade da câmera
def modify_camera_speed(camera, increase=True):
    original_speed = camera.speed
    if increase == False:
        original_speed *= -1
    def closure(*_):
        camera.speed += original_speed
    return closure

# Atribuição dos eventos de movimentação da câmera
window.add_key_callback("W", camera.move_forward)
window.add_key_callback("D", camera.move_right)
window.add_key_callback("S", camera.move_backward)
window.add_key_callback("A", camera.move_left)
window.add_cursor_callback(camera.move_front)
window.add_key_callback("Q", modify_camera_speed(camera, False))
window.add_key_callback("E", modify_camera_speed(camera, True))

# Execução

In [5]:
# Atributos de controle dos modelos
light_angle = 0.0
box_angle = 0.0

# Para transformar a luz
def transform_light():
    global light_angle 
    light_angle += 0.01
    tx = math.cos(light_angle) * 10
    ty = math.sin(light_angle) * 10
    tz = 3.0
    session.move_light(tx, ty, tz)
    light.model = translate(tx, ty, tz)

# Para transformar a caixa
def transform_box():
    global box_angle 
    box_angle += 1e-5
    box.model = xrotate(box_angle, box.model)

In [6]:
# Processamento dos buffers
session.process_buffers()

# Programa principal
for _ in window.display():

    # Limpeza da janela
    session.clear()

    # Renderização
    session.render()

    # Visualização
    session.view(camera.view)

    # Projeção
    session.projection(window.projection)